Penyelesaian studi kasus ekonometrika


• NRP mod 3 = 0: Model Regresi Linier Berganda dengan kasus multikolinieritas dan atau heteroskedastisitas, lakukan uji residual dan autokorelasi

• NRP mod 3 = 1: Model Regresi dengan Variabel Dummy

• NRP mod 3 = 2: Model Regresi Probit

Sumber Data: BPS, Data Jabar, Data Jakarta, BPS, Kaggle, dll.
File yang dikumpulkan: step by step penyelesaian studi kasus dalam pptx dan ppsx

### Buatlah Model Regresi Linier Berganda dengan kasus multikolinieritas dan atau heteroskedastisitas, lakukan uji residual dan autokorelasi

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import itertools
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt

# https://www.kaggle.com/datasets/arclaint/seki-bank-indonesia
df=pd.read_csv("D:/SDT_Semester_5/data_s5/SEKI_BI.csv")
df

,Date,BIRate_inPercentage,USPrimeRate_inPercentage,ExchangeRate_USDtoIDR,ForeignExchangeReserves_inMillionUSD,ForeignExchangeReserves_MonetaryGold_inMillionUSD,ForeignExchangeReserves_SDRs_inMillionUSD,ForeignExchangeReserves_RPF_inMillionUSD,ForeignExchangeReserves_Others_inMillionUSD,SuratUtangNegaraSUN_inMillionIDR,...,OutstandingDemandDeposit_Valas_inBillionIDR,OutstandingSavingDeposit_Rupiah_inBillionIDR,OutstandingSavingDeposit_Valas_inBillionIDR,OutstandingTimeDeposit_Rupiah_inBillionIDR,OutstandingTimeDeposit_Valas_inBillionIDR,SavingDepositInterestRate_Persero_inPercentage,SavingDepositInterestRate_Pemda_inPercentage,SavingDepositInterestRate_Private_inPercentage,SavingDepositInterestRate_Foreign_inPercentage,SavingDepositInterestRate_General_inPercentage
0,01/01/2010,6.50,3.25,9365,69562.47,2553.59,2743.60,226.50,64038.78,5.776650e+05,...,125045.3600,5.507960e+05,43074.4300,7.626724e+05,134470.8900,2.79,3.89,3.15,3.91,3.08
1,02/01/2010,6.50,3.25,9335,69730.60,2588.79,2699.23,222.80,64219.78,5.773030e+05,...,126392.5300,5.410685e+05,39990.8500,7.688128e+05,134454.5900,2.78,3.72,3.15,3.93,3.06
2,03/01/2010,6.50,3.25,9115,71823.21,2598.18,2678.19,221.10,66325.74,5.716490e+05,...,137589.7600,5.401521e+05,41453.3900,7.961739e+05,135731.2800,2.74,3.77,3.18,3.93,3.03
3,04/01/2010,6.50,3.25,9012,78582.77,2738.42,2658.47,219.50,72966.37,5.800190e+05,...,132910.1900,5.416820e+05,44369.3600,8.042122e+05,131346.4800,2.73,3.75,3.16,4.23,3.02
4,05/01/2010,6.50,3.25,9180,74587.29,2834.06,2598.36,214.50,68940.37,5.856660e+05,...,130873.0700,5.515772e+05,43165.4900,8.136474e+05,128769.8000,2.96,3.92,2.96,3.92,3.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,01/01/2024,6.00,8.50,15796,145052.00,5144.84,7399.99,1054.65,131452.09,4.667689e+06,...,755895.4150,2.492359e+06,175670.0449,2.712444e+06,329410.4663,0.61,0.86,0.66,2.66,0.68
169,02/01/2024,6.00,8.50,15673,144040.00,NaN,NaN,NaN,NaN,4.726066e+06,...,758442.2096,2.485649e+06,175702.1430,2.738829e+06,330561.2842,0.62,0.85,0.66,2.75,0.67
170,03/01/2024,6.00,8.50,15853,140390.00,NaN,NaN,NaN,NaN,4.648698e+06,...,760931.7679,2.531482e+06,176685.8079,2.783320e+06,334681.4287,0.63,0.90,0.65,2.76,0.68
171,04/01/2024,6.25,8.50,16249,136218.00,NaN,NaN,NaN,NaN,4.644098e+06,...,782879.7390,2.547993e+06,182073.3900,2.799113e+06,326482.8143,0.62,0.93,0.64,2.93,0.68


In [2]:
nan_counts = df.isna().sum()
zero_counts = (df == 0).sum()

print(df.dtypes)
print("Jumlah NaN per kolom:\n", nan_counts)
print("Jumlah nol per kolom:\n", zero_counts)

Date                                               object
BIRate_inPercentage                               float64
USPrimeRate_inPercentage                          float64
ExchangeRate_USDtoIDR                               int64
ForeignExchangeReserves_inMillionUSD              float64
                                                   ...   
SavingDepositInterestRate_Persero_inPercentage    float64
SavingDepositInterestRate_Pemda_inPercentage      float64
SavingDepositInterestRate_Private_inPercentage    float64
SavingDepositInterestRate_Foreign_inPercentage    float64
SavingDepositInterestRate_General_inPercentage    float64
Length: 72, dtype: object
Jumlah NaN per kolom:
 Date                                              0
BIRate_inPercentage                               0
USPrimeRate_inPercentage                          0
ExchangeRate_USDtoIDR                             0
ForeignExchangeReserves_inMillionUSD              0
                                                 ..
S

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
# Menghapus kolom yang tidak diperlukan
df = df.drop(columns=['Date'])

# Remove rows with NaN values
df = df.dropna()

# Step 2: Remove rows where all values are zero
# The (df_cleaned != 0).any(axis=1) checks if any value in the row is non-zero
df = df[(df != 0).any(axis=1)]

# Mengonversi kolom 'exp_percap' dari string ke float
df[df.select_dtypes(include='int').columns] = df.select_dtypes(include='int').astype(float)
df = df.apply(pd.to_numeric, errors='coerce')

# Normalisasi dengaN scaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled

,BIRate_inPercentage,USPrimeRate_inPercentage,ExchangeRate_USDtoIDR,ForeignExchangeReserves_inMillionUSD,ForeignExchangeReserves_MonetaryGold_inMillionUSD,ForeignExchangeReserves_SDRs_inMillionUSD,ForeignExchangeReserves_RPF_inMillionUSD,ForeignExchangeReserves_Others_inMillionUSD,SuratUtangNegaraSUN_inMillionIDR,SuratBerhargaSyariahNegaraSBSN_inMillionIDR,...,OutstandingDemandDeposit_Valas_inBillionIDR,OutstandingSavingDeposit_Rupiah_inBillionIDR,OutstandingSavingDeposit_Valas_inBillionIDR,OutstandingTimeDeposit_Rupiah_inBillionIDR,OutstandingTimeDeposit_Valas_inBillionIDR,SavingDepositInterestRate_Persero_inPercentage,SavingDepositInterestRate_Pemda_inPercentage,SavingDepositInterestRate_Private_inPercentage,SavingDepositInterestRate_Foreign_inPercentage,SavingDepositInterestRate_General_inPercentage
0,0.651656,-0.598165,-1.475830,-2.848624,-1.637450,-0.164544,-1.131564,-3.018654,-1.074554,-1.151426,...,-1.299025,-1.588577,-1.687972,-1.813191,-1.932432,2.415273,2.276498,1.958176,2.998272,2.218416
1,0.651656,-0.598165,-1.489144,-2.838722,-1.587150,-0.185556,-1.140127,-3.006871,-1.074837,-1.127187,...,-1.290313,-1.605649,-1.761537,-1.802415,-1.932680,2.398580,2.071367,1.958176,3.022404,2.189686
2,0.651656,-0.598165,-1.586782,-2.715479,-1.573732,-0.195520,-1.144061,-2.869769,-1.079265,-1.114090,...,-1.217901,-1.607257,-1.726645,-1.754398,-1.913314,2.331807,2.131700,1.997525,3.022404,2.146591
3,0.651656,-0.598165,-1.632494,-2.317378,-1.373329,-0.204859,-1.147764,-2.437452,-1.072710,-1.108825,...,-1.248164,-1.604572,-1.657079,-1.740291,-1.979825,2.315114,2.107567,1.971292,3.384381,2.132226
4,0.651656,-0.598165,-1.557935,-2.552690,-1.236660,-0.233324,-1.159336,-2.699552,-1.068288,-1.100830,...,-1.261338,-1.587206,-1.685800,-1.723733,-2.018909,2.699057,2.312697,1.708960,3.010338,2.175321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,0.066896,2.993863,1.258477,0.996807,1.475776,1.991930,0.750577,0.740854,2.016770,2.001300,...,2.242719,1.701040,1.376759,1.502227,0.763220,-1.240529,-1.452049,-1.294740,0.983265,-1.257915
165,0.261816,2.993863,1.431562,0.895744,1.918522,1.992986,0.751318,0.608774,2.021057,2.031108,...,2.480811,1.711722,1.333643,1.534364,0.779160,-1.223836,-1.452049,-1.307856,1.055660,-1.243550
166,0.261816,2.993863,1.195456,1.187861,2.094674,2.057405,0.796819,0.913539,2.095754,1.968720,...,2.606020,1.761678,1.369998,1.565290,0.715926,-1.223836,-1.415850,-1.307856,1.393506,-1.243550
167,0.261816,2.993863,1.209658,1.675743,2.193389,2.071029,0.806308,1.446184,2.131311,2.008364,...,2.817087,1.867368,1.370644,1.580690,0.930872,-1.223836,-1.258985,-1.294740,1.309044,-1.214820


In [7]:
import statsmodels.api as sm

x = df[['BIRate_inPercentage', 
         'USPrimeRate_inPercentage', 
         'ExchangeRate_USDtoIDR', 
         'ForeignExchangeReserves_MonetaryGold_inMillionUSD', 
         'ForeignExchangeReserves_SDRs_inMillionUSD', 
         'ForeignExchangeReserves_RPF_inMillionUSD', 
         'ForeignExchangeReserves_Others_inMillionUSD', 
         'SuratUtangNegaraSUN_inMillionIDR', 
         'SuratBerhargaSyariahNegaraSBSN_inMillionIDR', 
         'OutstandingDemandDeposit_Valas_inBillionIDR', 
         'OutstandingSavingDeposit_Rupiah_inBillionIDR', 
         'OutstandingSavingDeposit_Valas_inBillionIDR', 
         'OutstandingTimeDeposit_Rupiah_inBillionIDR', 
         'OutstandingTimeDeposit_Valas_inBillionIDR']]

# Pilih variabel dependen (Y)
y = df['ForeignExchangeReserves_inMillionUSD'] 

# Menambahkan konstanta untuk intercept
x = sm.add_constant(x)

# Membuat model regresi linier
model = sm.OLS(y, x).fit()

# Menampilkan ringkasan model
print(model.summary())



                                     OLS Regression Results                                     
Dep. Variable:     ForeignExchangeReserves_inMillionUSD   R-squared:                       1.000
Model:                                              OLS   Adj. R-squared:                  1.000
Method:                                   Least Squares   F-statistic:                 6.469e+04
Date:                                  Sun, 29 Sep 2024   Prob (F-statistic):          2.10e-282
Time:                                          02:40:40   Log-Likelihood:                -1152.4
No. Observations:                                   169   AIC:                             2335.
Df Residuals:                                       154   BIC:                             2382.
Df Model:                                            14                                         
Covariance Type:                              nonrobust                                         
                              

In [10]:
# VIF for each feature
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = x.columns
vif_data["VIF"] = [variance_inflation_factor(x.values, i)
                   for i in range(len(x.columns))]

print(vif_data)

                                              feature          VIF
0                                               const  1177.603944
1                                 BIRate_inPercentage     7.579083
2                            USPrimeRate_inPercentage     5.095582
3                               ExchangeRate_USDtoIDR    68.006458
4   ForeignExchangeReserves_MonetaryGold_inMillionUSD     9.838882
5           ForeignExchangeReserves_SDRs_inMillionUSD     7.593277
6            ForeignExchangeReserves_RPF_inMillionUSD    16.834770
7         ForeignExchangeReserves_Others_inMillionUSD    10.032031
8                    SuratUtangNegaraSUN_inMillionIDR   395.249545
9         SuratBerhargaSyariahNegaraSBSN_inMillionIDR   331.738718
10        OutstandingDemandDeposit_Valas_inBillionIDR    50.912044
11       OutstandingSavingDeposit_Rupiah_inBillionIDR   128.563023
12        OutstandingSavingDeposit_Valas_inBillionIDR    51.821888
13         OutstandingTimeDeposit_Rupiah_inBillionIDR   185.45

## Solusi Multiko dengan memperbaiki VIF

In [11]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

#scale predictor variables
pca = PCA()
xr=pca.fit_transform(scale(x))

#Splitting the dataset
#from skleaco.model selection import traio test split
#xr_train, xr_test, yr_train, yr_test = train_test_split(xr, y, test size = 0.3, random_state = 100)

# calculating VIF for each feature
from statsmodels.stats.outliers_influence import variance_inflation_factor

#VIF dataframe
xr = pd.DataFrame(xr)
vif_data_r = pd.DataFrame()
vif_data_r["feature"] = x.columns
vif_data_r["VIF"] = [variance_inflation_factor(xr.values, i) for i in range(len(xr.columns))]

print(vif_data_r)

                                              feature  VIF
0                                               const  1.0
1                                 BIRate_inPercentage  1.0
2                            USPrimeRate_inPercentage  1.0
3                               ExchangeRate_USDtoIDR  1.0
4   ForeignExchangeReserves_MonetaryGold_inMillionUSD  1.0
5           ForeignExchangeReserves_SDRs_inMillionUSD  1.0
6            ForeignExchangeReserves_RPF_inMillionUSD  1.0
7         ForeignExchangeReserves_Others_inMillionUSD  1.0
8                    SuratUtangNegaraSUN_inMillionIDR  1.0
9         SuratBerhargaSyariahNegaraSBSN_inMillionIDR  1.0
10        OutstandingDemandDeposit_Valas_inBillionIDR  1.0
11       OutstandingSavingDeposit_Rupiah_inBillionIDR  1.0
12        OutstandingSavingDeposit_Valas_inBillionIDR  1.0
13         OutstandingTimeDeposit_Rupiah_inBillionIDR  1.0
14          OutstandingTimeDeposit_Valas_inBillionIDR  NaN


c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


In [12]:
# import statsmodels.api as sm
xr = sm.add_constant(xr)
modelPCA = sm.OLS(y,xr).fit()
print(modelPCA.summary())

                                     OLS Regression Results                                     
Dep. Variable:     ForeignExchangeReserves_inMillionUSD   R-squared:                       1.000
Model:                                              OLS   Adj. R-squared:                  1.000
Method:                                   Least Squares   F-statistic:                 6.469e+04
Date:                                  Sun, 29 Sep 2024   Prob (F-statistic):          2.10e-282
Time:                                          02:43:01   Log-Likelihood:                -1152.4
No. Observations:                                   169   AIC:                             2335.
Df Residuals:                                       154   BIC:                             2382.
Df Model:                                            14                                         
Covariance Type:                              nonrobust                                         
                 coef    std e

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\regression\linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


In [13]:
import pandas as pd
import matplotlib.pyplot as plt

# Prediksi dari model
y_pred = modelPCA.predict(x)

# Hitung error antara actual dan predicted
error = y - y_pred

# Membuat DataFrame untuk actual, predicted, dan error
gab = pd.DataFrame({'Actual': y, 'Predicted': y_pred, 'Error': error})

# Menampilkan DataFrame
print(gab.tail(100))  # Menampilkan 100 data terakhir

# Plotting actual dan predicted dalam line chart untuk 100 data terakhir
plt.figure(figsize=(10,6))

# Plot actual values untuk 100 data terakhir
plt.plot(gab['Actual'].tail(100), label='Actual', color='blue', linewidth=2)

# Plot predicted values untuk 100 data terakhir
plt.plot(gab['Predicted'].tail(100), label='Predicted', color='orange', linewidth=2)

# Menambahkan label, judul, dan legend
plt.title('Actual vs Predicted Values (100 Last Data Points)')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()

# Menampilkan grafik
plt.show()


ValueError: shapes (169,15) and (16,) not aligned: 15 (dim 1) != 16 (dim 0)

In [ ]:
#Prediction
gab = pd.DataFrame({'Actual': y, 'Predicted':y_pred, 'Error': error})
gab 

In [ ]:
residuals = y - y_pred  # Menghitung residual

# Plot residuals
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred, y=residuals)
plt.axhline(0, color='orange', linestyle='--')
plt.title('Residual Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# Model Evaluation
from sklearn import metrics
meanAbErr = metrics.mean_absolute_error(y,y_pred)
MeanSqErr = metrics.mean_squared_error(y,y_pred)
rootMeanSqErr = np.sqrt(metrics.mean_squared_error(y, y_pred))
print('Mean Absolute Error:', meanAbErr)
print('Mean Square Error:', MeanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)